# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini Project Notebook: Employee Attrition Prediction

## **Note:** This notebook is part of an in-house Kaggle competition

## Problem Statement

To predict employee attrition using CatBoost and XgBoost

## Learning Objectives

At the end of the experiment, you will be able to

* explore the employee attrition dataset
* apply CatBoost and XgBoost on the dataset
* tune the model hyperparameters to improve accuracy
* evaluate the model using suitable metrics


## Introduction

Employee attrition is the gradual reduction in employee numbers. Employee attrition happens when the size of your workforce diminishes over time. This means that employees are leaving faster than they are hired. Employee attrition happens when employees retire, resign, or simply aren't replaced.
Although employee attrition can be company-wide, it may also be confined to specific parts of a business.

Employee attrition can happen for several reasons. These include unhappiness about employee benefits or the pay structure, a lack of employee development opportunities, and even poor conditions in the workplace.

To know more about the factors that lead to employee attrition, refer [here](https://www.betterup.com/blog/employee-attrition#:~:text=Employee%20attrition%20is%20the%20gradual,or%20simply%20aren't%20replaced).


**Gradient Boosted Decision Trees**

* Gradient boosted decision trees (GBDTs) are one of the most important machine learning models.

* GBDTs originate from AdaBoost, an algorithm that ensembles weak learners and uses the majority vote, weighted by their individual accuracy, to solve binary classification problems. The weak learners in this case are decision trees with a single split, called decision stumps.

* Some of the widely used gradient boosted decision trees are XgBoost, CatBoost and LightGBM.

## Dataset

The dataset used for this mini-project is [HR Employee Attrition dataset](https://data.world/aaizemberg/hr-employee-attrition). This dataset is synthetically created by IBM data scientists. There are 35 features and 1470 records.

There are numerical features such as:

* Age
* DistanceFromHome
* EmployeeNumber
* PerformanceRating

There are several categorical features such as:
* JobRole
* EducationField
* Department
* BusinessTravel

Dependent or target feature is 'attrition' which has values as Yes/No.

### **Kaggle Competition**

Please refer to the link for viewing the
[Kaggle Competition Document](https://drive.google.com/file/d/1c7PrbKrURFcnEB61dSoS9cBnUUVhhj-l/view?usp=drive_link) and join the Kaggle Competition using the hyperlink given in this document under '*Kaggle* Competition site'.

## Grading = 10 Points

In [1]:
!uv add ipykernel numpy matplotlib scikit-learn seaborn catboost scipy==1.12 xgboost lightgbm --upgrade


Resolved 58 packages in 576ms                                        
Audited 52 packages in 0.59ms


In [2]:
# @title Download the data
from utility import download_if_missing

download_if_missing(
    filename="hr_employee_attrition_train.csv",
    url="https://cdn.iisc.talentsprint.com/CDS/MiniProjects/hr_employee_attrition_train.csv",
)

# !wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/hr_employee_attrition_train.csv
print("Data Downloaded Successfuly!!")


Data Downloaded Successfuly!!


### Install CatBoost

In [3]:
# !pip -qq install catboost
# !uv add catboost numpy --upgrade


### Import Required Packages

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, metrics
import warnings

warnings.filterwarnings("ignore")
plt.style.use("fivethirtyeight")
pd.set_option("display.max_columns", 100)
%matplotlib inline


## Load the Dataset

**Exercise 1: Read the dataset [0.5 Mark]**

**Hint:** pd.read_csv()

In [5]:
# read the dataset
# YOUR CODE HERE

df = pd.read_csv("hr_employee_attrition_train.csv")


In [6]:
# Check the shape of dataframe.
# YOUR CODE HERE

df.shape


(1170, 35)

There can be more than one file to read as this is introduced as a competition, dataset has one file for training the model. Their can be other files as one containing the test features and the other can be the true labels.

## Data Exploration

- Check for missing values
- Check for consistent data type across a feature
- Check for outliers or inconsistencies in data columns
- Check for correlated features
- Do we have a target label imbalance
- How our independent variables are distributed relative to our target label
- Are there features that have strong linear or monotonic relationships? Making correlation heatmaps makes it easy to identify possible collinearity

**Exercise 2: Create a `List` of numerical and categorical columns. Display a statistical description of the dataset. Remove missing values (if any) [0.5 Mark]**

**Hint:** Use `for` to iterate through each column.

In [8]:
# Check the name and the data types of columns.
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   age                       1170 non-null   int64 
 1   businesstravel            1170 non-null   object
 2   dailyrate                 1170 non-null   int64 
 3   department                1170 non-null   object
 4   distancefromhome          1170 non-null   int64 
 5   education                 1170 non-null   int64 
 6   educationfield            1170 non-null   object
 7   employeecount             1170 non-null   int64 
 8   employeenumber            1170 non-null   int64 
 9   environmentsatisfaction   1170 non-null   int64 
 10  gender                    1170 non-null   object
 11  hourlyrate                1170 non-null   int64 
 12  jobinvolvement            1170 non-null   int64 
 13  joblevel                  1170 non-null   int64 
 14  jobrole                 

In [39]:
# check a random sample from the dataframe
df.sample(10)


,age,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,attrition
113,39,Travel_Rarely,408,Research & Development,2,4,Technical Degree,1,721,4,Female,80,2,2,Healthcare Representative,3,Single,4553,20978,1,Y,No,11,3,1,80,0,20,4,3,20,7,11,10,No
257,25,Travel_Rarely,1219,Research & Development,4,1,Technical Degree,1,1106,4,Male,32,3,1,Laboratory Technician,4,Married,3691,4605,1,Y,Yes,15,3,2,80,1,7,3,4,7,7,5,6,Yes
112,41,Travel_Rarely,645,Sales,1,3,Marketing,1,534,2,Male,49,4,3,Sales Executive,1,Married,8392,19566,1,Y,No,16,3,3,80,1,10,2,3,10,7,0,7,No
72,30,Travel_Rarely,288,Research & Development,2,3,Life Sciences,1,117,3,Male,99,2,2,Healthcare Representative,4,Married,4152,15830,1,Y,No,19,3,1,80,3,11,3,3,11,10,10,8,No
373,46,Travel_Rarely,1485,Research & Development,18,3,Medical,1,550,3,Female,87,3,2,Manufacturing Director,3,Divorced,4810,26314,2,Y,No,14,3,3,80,1,19,5,2,10,7,0,8,No
819,34,Travel_Rarely,479,Research & Development,7,4,Medical,1,1577,1,Male,35,3,1,Research Scientist,4,Single,2972,22061,1,Y,No,13,3,3,80,0,1,4,1,1,0,0,0,No
999,50,Travel_Rarely,1207,Research & Development,28,1,Medical,1,716,4,Male,74,4,1,Laboratory Technician,3,Married,3221,3297,1,Y,Yes,11,3,3,80,3,20,3,3,20,8,3,8,No
922,22,Travel_Rarely,581,Research & Development,1,2,Life Sciences,1,2007,4,Male,63,3,1,Research Scientist,3,Single,3375,17624,0,Y,No,12,3,4,80,0,4,2,4,3,2,1,2,No
336,44,Travel_Rarely,477,Research & Development,7,4,Medical,1,36,1,Female,42,2,3,Healthcare Representative,4,Married,10248,2094,3,Y,No,14,3,4,80,1,24,4,3,22,6,5,17,No
1015,43,Travel_Frequently,559,Research & Development,10,4,Life Sciences,1,448,3,Female,82,2,2,Laboratory Technician,3,Divorced,5257,6227,1,Y,No,11,3,2,80,1,9,3,4,9,7,0,0,No


In [40]:
# separate the columns as categorical and numerical
categorical_columns = df.select_dtypes(include=["object"]).columns
numerical_columns = df.select_dtypes(exclude=["object"]).columns

categorical_columns = [
    "businesstravel",
    "department",
    "education",
    "educationfield",
    "environmentsatisfaction",
    "gender",
    "jobinvolvement",
    "joblevel",
    "jobrole",
    "jobsatisfaction",
    "maritalstatus",
    "over18",
    "overtime",
    "standardhours",
    "performancerating",
    "relationshipsatisfaction",
    "stockoptionlevel",
    "worklifebalance",
    "attrition",
]
numerical_columns = [
    "age",
    "dailyrate",
    "distancefromhome",
    "hourlyrate",
    "monthlyincome",
    "monthlyrate",
    "numcompaniesworked",
    "percentsalaryhike",
    "totalworkingyears",
    "trainingtimeslastyear",
    "yearsatcompany",
    "yearsincurrentrole",
    "yearssincelastpromotion",
    "yearswithcurrmanager",
]
useless_columns = ["employeecount", "employeenumber"]

print(f"Number of Categorical Columns: {len(categorical_columns)}")
print(f"Number of Numerical Columns: {len(numerical_columns)}")
print(f"Number of Useless Columns: {len(useless_columns)}")
print(
    f"Total Columns: {len(categorical_columns) + len(numerical_columns) + len(useless_columns)}"
)


Number of Categorical Columns: 19
Number of Numerical Columns: 14
Number of Useless Columns: 2
Total Columns: 35


In [41]:
# Analyze categorical columns in a dataset
## unique values in each column
## count of unique values in each column
## missing values in each column

# Create a dataframe to store the information
df_categorical_info = pd.DataFrame(
    columns=["Column", "Unique Values", "Count of Unique Values", "Missing Values"]
)

# Iterate through each categorical column and collect the required information
for col in categorical_columns:
    unique_values = df[col].unique()
    count_unique_values = df[col].nunique()
    missing_values = df[col].isnull().sum()
    unique_val = ", ".join(unique_values.astype(str))
    # print(un_val)
    # Append the information to the dataframe
    temp_df = pd.DataFrame(
        {
            "Column": col,
            "Unique Values": [unique_val],
            "Count of Unique Values": count_unique_values,
            "Missing Values": missing_values,
        }
    )

    df_categorical_info = pd.concat([df_categorical_info, temp_df], axis=0)
# Display the dataframe
df_categorical_info


,Column,Unique Values,Count of Unique Values,Missing Values
0,businesstravel,"Travel_Rarely, Travel_Frequently, Non-Travel",3,0
0,department,"Research & Development, Sales, Human Resources",3,0
0,education,"2, 3, 4, 5, 1",5,0
0,educationfield,"Life Sciences, Medical, Technical Degree, Othe...",6,0
0,environmentsatisfaction,"2, 3, 4, 1",4,0
0,gender,"Female, Male",2,0
0,jobinvolvement,"2, 3, 1, 4",4,0
0,joblevel,"2, 1, 3, 4, 5",5,0
0,jobrole,"Manufacturing Director, Healthcare Representat...",9,0
0,jobsatisfaction,"4, 3, 1, 2",4,0


In [42]:
# Analyze numerical columns in a dataset
## mean, median, mode, min, max, range, standard deviation, variance
## missing values in each column

# Create a dataframe to store the information
df_numerical_info = pd.DataFrame(
    columns=[
        "Column",
        "Mean",
        "Median (50th Percentile)",
        "Mode",
        "Min",
        "Max",
        "Range",
        "Standard Deviation",
        "Variance",
        "25th Percentile",
        "50th Percentile (Median)",
        "75th Percentile",
        "Count of Unique Values",
        "Missing Values",
    ]
)

# Iterate through each categorical column and collect the required information
for col in numerical_columns:
    mean = df[col].mean()
    median = df[col].median()
    mode = df[col].mode()[0]
    minimum = df[col].min()
    maximum = df[col].max()
    range_val = maximum - minimum
    std_dev = df[col].std()
    variance = df[col].var()
    missing_values = df[col].isnull().sum()
    percentile_25 = df[col].quantile(0.25)
    percentile_50 = df[col].quantile(0.50)
    percentile_75 = df[col].quantile(0.75)
    count_unique_values = df[col].nunique()
    # Append the information to the dataframe
    temp_df = pd.DataFrame(
        {
            "Column": col,
            "Mean": [mean],
            "Median (50th Percentile)": [median],
            "Mode": [mode],
            "Min": [minimum],
            "Max": [maximum],
            "Range": [range_val],
            "Standard Deviation": [std_dev],
            "Variance": [variance],
            "25th Percentile": [percentile_25],
            "50th Percentile (Median)": [percentile_50],
            "75th Percentile": [percentile_75],
            "Count of Unique Values": count_unique_values,
            "Missing Values": [missing_values],
        }
    )

    df_numerical_info = pd.concat([df_numerical_info, temp_df], axis=0)

# Display the dataframe
df_numerical_info


,Column,Mean,Median (50th Percentile),Mode,Min,Max,Range,Standard Deviation,Variance,25th Percentile,50th Percentile (Median),75th Percentile,Count of Unique Values,Missing Values
0,age,36.858120,35.0,34,18,60,42,9.183448,8.433571e+01,30.00,35.0,43.00,43,0
0,dailyrate,797.822222,798.0,408,102,1499,1397,403.592661,1.628870e+05,461.00,798.0,1146.75,778,0
0,distancefromhome,9.262393,7.0,1,1,29,28,8.150868,6.643665e+01,2.00,7.0,14.00,29,0
0,hourlyrate,65.501709,66.0,42,30,100,70,20.310054,4.124983e+02,48.00,66.0,83.00,71,0
0,monthlyincome,6456.937607,4903.5,2380,1009,19999,18990,4660.476506,2.172004e+07,2914.75,4903.5,8215.25,1097,0
0,monthlyrate,14411.584615,14536.0,9150,2094,26999,24905,7078.167743,5.010046e+07,8387.50,14536.0,20456.25,1149,0
0,numcompaniesworked,2.750427,2.0,1,0,9,9,2.521952,6.360244e+00,1.00,2.0,4.00,10,0
0,percentsalaryhike,15.204274,14.0,11,11,25,14,3.688952,1.360836e+01,12.00,14.0,18.00,15,0
0,totalworkingyears,11.315385,10.0,10,0,40,40,7.879363,6.208437e+01,6.00,10.0,15.00,40,0
0,trainingtimeslastyear,2.842735,3.0,2,0,6,6,1.306385,1.706641e+00,2.00,3.0,3.00,7,0


In [43]:
# Analyze useless columns in the dataset

df_useless_info = pd.DataFrame(
    columns=["Column", "Unique Values", "Count of Unique Values", "Missing Values"]
)

# Iterate through each useless column and collect the required information
for col in useless_columns:
    unique_values = df[col].unique()
    count_unique_values = df[col].nunique()
    missing_values = df[col].isnull().sum()
    unique_val = ", ".join(unique_values.astype(str))
    temp_df = pd.DataFrame(
        {
            "Column": col,
            "Unique Values": [unique_val],
            "Count of Unique Values": count_unique_values,
            "Missing Values": missing_values,
        }
    )

    df_useless_info = pd.concat([df_useless_info, temp_df], axis=0)
# Display the dataframe
df_useless_info


,Column,Unique Values,Count of Unique Values,Missing Values
0,employeecount,1,1,0
0,employeenumber,"1888, 757, 1310, 476, 1361, 565, 482, 1506, 24...",1170,0


#### Observations
1. `over18`, `standardhours`: has a single value in the entire dataset. Hence needs to check if test test has any other value? Should I drop? 
1. `employeecount`: has a single repeating values in the entire table. Should be dropped.
1. `employeenumber`: is just a serial number. Should be dropped.
1. `trainingtimeslastyear`, `numcompaniesworked`: can we move this to categorical variable as they have 7 and 10 values respectively.
1. Outliers:
    1. 

In [7]:
# YOUR CODE HERE
df.describe()


,age,dailyrate,distancefromhome,education,employeecount,employeenumber,environmentsatisfaction,hourlyrate,jobinvolvement,joblevel,jobsatisfaction,monthlyincome,monthlyrate,numcompaniesworked,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager
count,1170.000000,1170.000000,1170.000000,1170.000000,1170.0,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.0,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.00000,1170.000000,1170.000000
mean,36.858120,797.822222,9.262393,2.917949,1.0,1021.113675,2.723932,65.501709,2.747009,2.049573,2.723077,6456.937607,14411.584615,2.750427,15.204274,3.153846,2.739316,80.0,0.785470,11.315385,2.842735,2.745299,6.925641,4.17094,2.138462,4.061538
std,9.183448,403.592661,8.150868,1.011534,0.0,600.548289,1.095847,20.310054,0.702625,1.096352,1.103799,4660.476506,7078.167743,2.521952,3.688952,0.360955,1.075225,0.0,0.847253,7.879363,1.306385,0.718864,6.232076,3.62788,3.201738,3.586350
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,1009.000000,2094.000000,0.000000,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000
25%,30.000000,461.000000,2.000000,2.000000,1.0,496.250000,2.000000,48.000000,2.000000,1.000000,2.000000,2914.750000,8387.500000,1.000000,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.00000,0.000000,2.000000
50%,35.000000,798.000000,7.000000,3.000000,1.0,1008.500000,3.000000,66.000000,3.000000,2.000000,3.000000,4903.500000,14536.000000,2.000000,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.00000,1.000000,3.000000
75%,43.000000,1146.750000,14.000000,4.000000,1.0,1548.500000,4.000000,83.000000,3.000000,3.000000,4.000000,8215.250000,20456.250000,4.000000,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.00000,2.750000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2065.000000,4.000000,100.000000,4.000000,5.000000,4.000000,19999.000000,26999.000000,9.000000,25.000000,4.000000,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.00000,15.000000,17.000000


First, we want to get a sense of our data:
- What features have the most divergent distributions based on target class
- Do we have a target label imbalance
- How our independent variables are distributed relative to our target label
- Are there features that have strong linear or monotonic relationships, making correlation heatmaps makes it easy to identify possible colinearity

### Check for outliers

**Exercise 3: Create a box plot to check for outliers [0.5 Mark]**

In [ ]:
# Check for outliers
# YOUR CODE HERE


### Handling outliers

**Exercise 4: Use lower bound as 25% and upper bound as 75% to handle the outliers [0.5 Mark]**

In [ ]:
# YOUR CODE HERE


In [ ]:
# Recheck for outliers
# YOUR CODE HERE


### Target label imbalance

**Exercise 5: Check if there is an imbalance in target label [0.5 Mark]**

**Hint:** Use value_counts()

In [ ]:
# Count of unique values in Attrition column
# YOUR CODE HERE


In [ ]:
# Plot barplot to visualize balance/imbalance
# YOUR CODE HERE


If there is any imbalance in the dataset then a few techniques can be utilised (optional):
1. SMOTE
2. Cross Validation
3. Regularizing the model's parameters

###Plot pairplot

**Exercise 6: Visualize the relationships between the predictor variables and the target variable using a pairplot [0.5 Mark]**

**Hint:** Use sns.pairplot

In [ ]:
# Visualize a pairplot with relevant features
# YOUR CODE HERE


### Explore Correlation

- Plotting the Heatmap

**Exercise 7: Visualize the correlation among IBM employee attrition numerical features using a heatmap [0.5 Mark]**

In [ ]:
# Visualize heatmap
# YOUR CODE HERE


Comment on the observations made with the pairplot and heatmap

### Preparing the test feature space
* Remove outliers if any
* Handle the categorical feature if required
* Other processing steps can also be followed.

In [ ]:
# YOUR CODE HERE


Optional:
Use `Hyperopt`, a hyperparameter tuning technique to identify the best set of parameters.

Refer to the Additional Notebook: CatBoost parameter tuning [CDS-B8 GDrive -> Module 3 -> Assignments -> July 27, 2024 -> Additional Notebook (ungraded) -> Addl_NB_Tuning_hyerparameters_using_Hyperopt]

In the notebook, data processing is done separately for different models.
Considering the fact that different models may require data in different format and in turn different processes may be followed to process the data.

If the processing steps followed for the models are same, data processing can also be done once.

## Apply CatBoost

Catboost was released in 2017 by Yandex, showing, by their benchmark to be faster in prediction, better in accuracy, and easier to use for categorical data across a series of GBDT tasks. Additional capabilities of catboost include plotting feature interactions and object (row) importance.

[Here](https://catboost.ai/en/docs/) is the official documentation of CatBoost

### Data Processing for CatBoost

**Exercise 8: Data processing for CatBoost [1 Mark]**
* **Copy the dataframe that was created after removing the outliers**
* **Handle the categorical features if required**
* **Create target column and feature space**

**Hint:** Column containing the information on attrition will be the target column.

In [ ]:
# Copy the data
# YOUR CODE HERE


In [ ]:
# Target Column
# YOUR CODE HERE


In [ ]:
# Feature Space
# YOUR CODE HERE


### Model Definition

**Exercise 9: Define, train the model and display the results [2 Mark]**

**Hint:**
* Use CatBoostClassifier() to define the model with relevant parameters.
* Use `fit` to fit the data to the model. Refer [here](https://catboost.ai/en/docs/concepts/speed-up-training) to see some ways to speedup CatBoost training.
* Evaluate the model using roc_auc_score, accuracy_score, f1_score, predict methods or other relevant techniques.

In [ ]:
# Create CatBoost model
# YOUR CODE HERE


In [ ]:
# Model training
# YOUR CODE HERE


### Model performance

In [ ]:
# Model performance on all sets
# YOUR CODE HERE


## Apply XGBoost

XGBoost is a workhorse gradient boosted decision tree algorithm. Its been around since 2014 and has come to dominate the Kaggle and data science community. XGB introduced gradient boosting where new models are fit to the residuals of prior models and then added together, using a gradient descent algorithm to minimize the loss.

Read [here](https://xgboost.readthedocs.io/en/stable/parameter.html) on XGBoost parameters.

Refer [here](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBClassifier) for the official documentation of XGBoost classifier.

### Data Processing for XGBoost


**Exercise 10: Data Processing for XGBoost [1 Mark]**
* **Copy the dataframe after the outliers were removed.**
* **Handle the categorical features if required**
* **Create target column and feature space**

In [ ]:
# Copy dataframe
# YOUR CODE HERE


**Hint:** Use pd.get_dummies

In [ ]:
# Handling categorical features
# YOUR CODE HERE


In [ ]:
# Concat the dummy variables to actual dataframe and remove initial categorical columns
# YOUR CODE HERE


When creating the dummy variables, the name of attrition column was changed, rename to 'attrition' again.

**Hint:** Use .rename

In [ ]:
# Rename target column
# YOUR CODE HERE


In [ ]:
# Feature Space
# YOUR CODE HERE

# Targer label
# YOUR CODE HERE


### Model Definition

**Exercise 11: Define, train the model and display the results [2 Mark]**

**Hint:**
* Use XGBClassifier() to define the model with relevant parameters.
* Use `fit` to fit the data to the model.
* Evaluate the model using roc_auc_score, accuracy_score, f1_score, predict methods or other relevant techniques.

In [ ]:
# Create XGBoost classifier model
# YOUR CODE HERE


In [ ]:
# Model training
# YOUR CODE HERE


### Model Performance

In [ ]:
# Model performance on all sets
# YOUR CODE HERE


## Apply LightGBM (Optional)

LightGBM is an open-source GBDT framework created by Microsoft as a fast and scalable alternative to XGB and GBM. By default LightGBM will train a Gradient Boosted Decision Tree (GBDT), but it also supports random forests, Dropouts meet Multiple Additive Regression Trees (DART), and Gradient Based One-Side Sampling (Goss).

To know more about LightGBM parameters, refer [here](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier).

### Feature Engineering for LightGBM

In [ ]:
## Following the same procedure as followed in XGBoost

# Copy the dataframe
# YOUR CODE HERE

# Handling categorical features
# YOUR CODE HERE

# Concat the dummy variables to actual dataframe and remove initial categorical columns
# YOUR CODE HERE

# Rename target column
# YOUR CODE HERE

# Features Space
# YOUR CODE HERE

# Target Label
# YOUR CODE HERE


### Model Definition

**Hint:**
* Use LGBMClassifier() to define the model with relevant parameters.
* Use `fit` to fit the data to the model.
* Evaluate the model using roc_auc_score, accuracy_score, f1_score, predict methods or other relevant techniques.

In [ ]:
# Create LightGBM classifier model
# YOUR CODE HERE


In [ ]:
# Model training
# YOUR CODE HERE


### Model performance

In [ ]:
# Model performance on all sets
# YOUR CODE HERE


## Results

**Exercise 12: Create a dataframe of XGBoost results and CatBoost results and display them [0.5 Mark]**

**Hint:** Use pd.DataFrame

In [ ]:
# Create a dataframe for computed metrics for different models
# YOUR CODE HERE


Reference reading:
1. https://machinelearningmastery.com/xgboost-for-imbalanced-classification/

## Kaggle Prediction

Load data from Kaggle competition site

In [ ]:
# From the given Kaggle competition link, load the dataset 'hr_employee_attrition_test.csv'
# YOUR CODE HERE


In [ ]:
# From the dataset 'hr_employee_attrition_test.csv', drop columns ['id','employeenumber', 'employeecount', 'over18'] having single value
# YOUR CODE HERE


In [ ]:
# Handle categorical features
# YOUR CODE HERE


In [ ]:
# Concat the dummy variables to actual dataframe and remove initial categorical columns
# YOUR CODE HERE


Predictions

In [ ]:
# Get the predictions using your already trained CatBoost classifier model achieved in Exercise 9
# YOUR CODE HERE


In [ ]:
# Get the predictions using your already trained XGBoost classifier model achieved in Exercise 11
# YOUR CODE HERE


Get the predictions using your trained Microsoft LightGBM model (Optional)

In [ ]:
# Get the predictions using your already trained Microsoft LightGBM classifier model
# achieved under the optional exercise 'Apply LightGBM (Optional)'
# YOUR CODE HERE


Save predictions to csv and submit under given Kaggle competiton link

In [ ]:
# YOUR CODE HERE
